In [64]:
from subprocess import call
call(["pip", "install", "unicode-slugify"])

0

In [8]:
import requests
from fake_useragent import UserAgent
import random
import time

PROXIES = [
#     'http://75.151.213.85:8080',
    'http://168.63.20.19:8122',
]

def requests_get(url, use_proxy=False):
    proxies = None
    if use_proxy:
        proxy = random.choice(PROXIES)
        proxies = {'http': proxy}
        
    ua = UserAgent().random
    
#     try:    
    return requests.get(url, headers={'User-Agent': ua}, proxies=proxies)
#     except:
#         return requests_get(url)

# First fetch Billboard top 100

Uses RSS feed reader in this format:

```
<item>
  <title>1: Bad And Boujee</title>
  <link>http://www.billboard.com/charts/hot-100</link>
  <artist>Migos Featuring Lil Uzi Vert</artist>
  <chart_item_title>Bad And Boujee</chart_item_title>
  <rank_this_week>1</rank_this_week>
  <rank_last_week>1</rank_last_week>
  <description>Bad And Boujee by Migos Featuring Lil Uzi Vert ranks #1</description>
  <category>The Hot 100</category>
  <pubDate>Sat, 11 Feb 2017 00:00:00 -0500</pubDate>
  <guid isPermaLink="true">http://www.billboard.com/charts/hot-100</guid>
 </item>
```

In [9]:
# Fetch the Billboard Hot 100 for this week

import requests
from lxml import etree

def get_billboard_hot_100():
    hot_100_rss = requests_get('http://www.billboard.com/rss/charts/hot-100')
    xml = bytes(bytearray(hot_100_rss.text, encoding='utf-8'))
    tree = etree.XML(xml)

    songs = []
    items = tree.xpath("//item")
    for item in items:
        song = {}

        song['title'] = item.xpath("chart_item_title/text()")[0]
        song['artist'] = item.xpath("artist/text()")[0]
        song['rank_this_week'] = item.xpath("rank_this_week/text()")[0]
        song['rank_last_week'] = item.xpath("rank_last_week/text()")[0]

        songs.append(song)
        
    return songs

songs = get_billboard_hot_100()
songs

[{'artist': 'Migos Featuring Lil Uzi Vert',
  'rank_last_week': '1',
  'rank_this_week': '1',
  'title': 'Bad And Boujee'},
 {'artist': 'Ed Sheeran',
  'rank_last_week': '2',
  'rank_this_week': '2',
  'title': 'Shape Of You'},
 {'artist': 'The Chainsmokers Featuring Halsey',
  'rank_last_week': '4',
  'rank_this_week': '3',
  'title': 'Closer'},
 {'artist': 'Machine Gun Kelly x Camila Cabello',
  'rank_last_week': '6',
  'rank_this_week': '4',
  'title': 'Bad Things'},
 {'artist': 'Zayn / Taylor Swift',
  'rank_last_week': '8',
  'rank_this_week': '5',
  'title': "I Don't Wanna Live Forever (Fifty Shades Darker)"},
 {'artist': 'Rae Sremmurd Featuring Gucci Mane',
  'rank_last_week': '3',
  'rank_this_week': '6',
  'title': 'Black Beatles'},
 {'artist': 'Maroon 5 Featuring Kendrick Lamar',
  'rank_last_week': '9',
  'rank_this_week': '7',
  'title': "Don't Wanna Know"},
 {'artist': 'The Weeknd Featuring Daft Punk',
  'rank_last_week': '5',
  'rank_this_week': '8',
  'title': 'Starboy'}

# Test scraping lyrics from AZLyrics.com

http://search.azlyrics.com/search.php?q=Bad+And+Boujee+Migos+Lil+Uzi+Vert

Initial code borrowed from:

https://github.com/blha303/getlyrics

This method works, however you must be cautious in how many requests you perform in a short time frame. Otherwise you might get your IP permanently blocked from the site.

In [7]:
from urllib.parse import urlencode
from lxml import html
from lxml.etree import tostring

def get_az_lyrics(search):   
    # search the AZLyrics site and scrape results
    base_url = 'http://search.azlyrics.com/search.php?'
    full_url = base_url + urlencode({'q': search})
    
    result = requests_get(full_url)
    tree = html.fromstring(result.text)
    
    # Look for all URLs in search results
    # When no results - return None
    search_results = tree.xpath("//td/a[@target='_blank']")
    if not search_results:
        return None
    
    # Scrape the lyrics page for the lyrics
    lyrics_url = search_results[0].get('href')
    result = requests_get(lyrics_url)
    tree = html.fromstring(result.text)
    
    lyrics_html = tree.xpath("//div")
    return lyrics_html[22].text_content()

lyrics = get_az_lyrics("Bad And Boujee Migos Lil Uzi Vert")
print(lyrics)

IndexError: list index out of range

In [ ]:
import time

missing = 0
total = 0

# Scrape all lyrics for all songs
for song in songs:
    search_criteria = song['title'] + ' ' + song['artist'].replace('Featuring', '')
    song['lyrics'] = get_az_lyrics(search_criteria)
    
    time.sleep(1)
    
    if not song['lyrics']:
        missing = missing + 1
        
    total = total + 1
    
    if total >= 5:
        break
        
print(missing)

# Scrape Lyrics from LyricsFreak

http://www.lyricsfreak.com/search.php?a=search&type=song&q=Bad+And+Boujee

This site only really lets you search by song or artist only. That is odd...


In [15]:
from urllib.parse import urlencode
from lxml import html
from lxml.etree import tostring
import re

def get_lyrics_freak(artist, song):
    def primary_artist(artist):
        artist = artist.casefold()
        artist = artist.split('featuring')[0].strip()
        artist = artist.split('/')[0].strip()
        artist = artist.split(' x ')[0].strip()

        return artist
    
    artist = primary_artist(artist)
    
    # search the LyricsFreak site and scrape results
    base_url = 'http://www.lyricsfreak.com'
    full_url = base_url + '/search.php?' + urlencode({'a': 'search', 'type': 'song', 'q': song})
    
    result = requests_get(full_url)
    tree = html.fromstring(result.text)
    
    # Look for all URLs in search results
    # When no results - return None
    search_results = tree.xpath("//td/a")
    if not search_results:
        return None
    
    # find the lyrics page...
    previous_is_artist = False
    lyrics_url = None
    for result in search_results:
        current_text = result.text_content().casefold()
        if previous_is_artist and song.casefold() in current_text:
            lyrics_url = base_url + result.get("href")
            break        
        
        previous_is_artist = artist.casefold() in current_text
    
    if not lyrics_url:
        return None
    
    # scrape the lyrics
    result = requests_get(lyrics_url)
    tree = html.fromstring(result.text)
    
    lyric_html = tree.xpath("//div[@id='content_h']")
    if not lyric_html:
        return None
    
    lyric_html = str(tostring(lyric_html[0]))
    return html.fromstring(re.sub(r'<br\w*\/?>', r'\\n', lyric_html)).text_content()

print(get_lyrics_freak("Migos", "Bad And Boujee"))

b'[Intro: Offset]\nYou know, young rich niggas\nYou know so we never really had no old money\nWe got a whole lotta new money though, hah\n(If Young Metro don\'t trust you I\'m gon\' shoot ya)\n\n[Hook: Offset]\nRaindrop, drop top (drop top)\nSmokin\' on cookie in the hotbox (cookie)\nFuckin\' on your bitch she a thot, thot (thot)\nCookin\' up dope in the crockpot (pot)\nWe came from nothin\' to somethin\' nigga (hey)\nI don\'t trust nobody grip the trigger (nobody)\nCall up the gang, and they come and get you (gang)\nCry me a river, give you a tissue (hey)\nMy bitch is bad and boujee (bad)\nCookin\' up dope with an Uzi (blaow)\nMy niggas is savage, ruthless (savage)\nWe got 30\'s and 100 rounds too (grrah)\nMy bitch is bad and boujee (bad)\nCookin\' up dope with an Uzi (dope)\nMy niggas is savage, ruthless (hey)\nWe got 30\'s and 100 rounds too (glah)\n\n[Verse 1: Offset]\nOffset, whoo, whoo, whoo, whoo, whoo!\nRackings on rackings, got backends on backends\nI\'m ridin\' around in a co

In [50]:
import time

missing = 0
total = 0

# Scrape all lyrics for all songs
    song['lyrics'] = get_lyrics_freak(song['artist'], song['title'])
    
    time.sleep(1)
    
    if not song['lyrics']:
        missing = missing + 1
        
print(missing)

migos
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Bad+And+Boujee
http://www.lyricsfreak.com/m/migos/bad+and+boujee_21112393.html
ed sheeran
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Shape+Of+You
None
the chainsmokers
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Closer
None
machine gun kelly
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Bad+Things
http://www.lyricsfreak.com/m/machine+gun+kelly/bad+things_21111487.html
zayn
http://www.lyricsfreak.com/search.php?type=song&a=search&q=I+Don%27t+Wanna+Live+Forever+%28Fifty+Shades+Darker%29
http://www.lyricsfreak.com/z/zayn+taylor+swift/i+dont+wanna+live+forever+fifty+shades+darker_21112195.html
rae sremmurd
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Black+Beatles
http://www.lyricsfreak.com/r/rae+sremmurd/black+beatles_21111299.html
maroon 5
http://www.lyricsfreak.com/search.php?type=song&a=search&q=Don%27t+Wanna+Know
http://www.lyricsfreak.com/m/maroon+5/dont+wanna+kn

In [54]:
for song in songs:
    if song['lyrics'] is None:
        print(song['artist'] + ' '+ song['title'])

Ed Sheeran Shape Of You
The Chainsmokers Featuring Halsey Closer
The Weeknd Featuring Daft Punk Starboy
The Chainsmokers Paris
Big Sean Bounce Back
DJ Snake Featuring Justin Bieber Let Me Love You
Amine Caroline
The Weeknd Featuring Daft Punk I Feel It Coming
Shawn Mendes Mercy
D.R.A.M. Featuring Lil Yachty Broccoli
KYLE Featuring Lil Yachty iSpy
Zay Hilfigerrr & Zayion McCall Juju On That Beat (TZ Anthem)
Clean Bandit Featuring Sean Paul & Anne-Marie Rockabye
The Chainsmokers Featuring Daya Don't Let Me Down
Little Big Town Better Man
Migos T-Shirt
Chris Brown Featuring Usher & Gucci Mane Party
21 Savage & Metro Boomin Featuring Future X
Blake Shelton A Guy With A Girl
Jon Pardi Dirt On My Boots
J. Cole Deja Vu
Future Featuring Drake Used To This
Gucci Mane Featuring Drake Both
The Weeknd Party Monster
Marian Hill Down
Kaleo Way Down We Go
Dustin Lynch Seein' Red
Bebe Rexha I Got You
Noah Cyrus Featuring Labrinth Make Me (Cry)
Post Malone Featuring Quavo Congratulations
21 Savage & Me

# Scrape Lyrics from SongLyrics

http://www.songlyrics.com/index.php?section=search&searchW=The+Chainsmokers+Featuring+Halsey+Closer&submit=Search&searchIn1=artist&searchIn3=song

In [12]:
from urllib.parse import urlencode
from lxml import html
from lxml.etree import tostring

def get_song_lyrics(artist, song):   
    # search the SongLyrics site and scrape results
    base_url = 'http://www.songlyrics.com'
    params = {
        'section': 'search',
        'searchW': artist + ' ' + song,
        'submit': 'Search',
        'searchIn1': 'artist',
        'searchIn3': 'song'
    }
    full_url = base_url + '/index.php?' + urlencode(params)
    
    result = requests_get(full_url)
    tree = html.fromstring(result.text)
    
    # Look for all URLs in search results
    # When no results - return None
    search_results = tree.xpath("//div[@class='serpresult']/h3/a")
    if not search_results:
        return None
    
    lyrics_url = search_results[0].get('href')
    
    if not lyrics_url:
        return None
    
    # scrape the lyrics
    result = requests_get(lyrics_url)
    tree = html.fromstring(result.text)
    
    lyric_html = tree.xpath("//p[@id='songLyricsDiv']")
    if not lyric_html:
        return None
    
    return lyric_html[0].text_content() 

print(get_song_lyrics("Migos", "Bad And Boujee"))

http://www.songlyrics.com/index.php?searchW=Migos+Bad+And+Boujee&searchIn1=artist&submit=Search&section=search&searchIn3=song
http://www.songlyrics.com/migos/bad-and-boujee-feat-lil-uzi-vert-lyrics/
Produced By Metro Boomin


[Intro: Offset]
You know young rich niggas
You know so we never really had no old money
We got a whole lotta new money though, hah
(If Young Metro don't trust you I'm gon' shoot ya)

[Hook: Offset]
Raindrops, drop tops (drop top)
Smokin' on cookie in the hotbox (cookie)
Fuckin' on your bitch she a thot, thot (thot)
Cookin' up dope in the crockpot, (pot)
We came from nothin' to somethin' nigga (hey)
I don't trust nobody grip the trigger (nobody)
Call up the gang, they come and get you (gang)
Cry me a river, give you a tissue (hey)
My bitch is bad and boujee (bad)
Cookin' up dope with an Uzi (blaow)
My niggas is savage, ruthless (savage)
We got 30's and 100 rounds too (grrah)
My bitch is bad and boujee (bad)
Cookin' up dope with an Uzi (dope)
My niggas is savage, ru

In [61]:
import time

missing = 0
total = 0

# Scrape all lyrics for all songs
for song in songs:
    song['lyrics'] = get_song_lyrics(song['artist'], song['title'])
    
    time.sleep(1)
    
    if not song['lyrics']:
        missing = missing + 1
        
print(missing)

http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=Migos+Featuring+Lil+Uzi+Vert+Bad+And+Boujee&submit=Search
http://www.songlyrics.com/migos/bad-and-boujee-feat-lil-uzi-vert-lyrics/
http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=Ed+Sheeran+Shape+Of+You&submit=Search
http://www.songlyrics.com/ed-sheeran/shape-of-you-lyrics/
http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=The+Chainsmokers+Featuring+Halsey+Closer&submit=Search
http://www.songlyrics.com/the-chainsmokers-feat-halsey/closer-feat-halsey-lyrics/
http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=Machine+Gun+Kelly+x+Camila+Cabello+Bad+Things&submit=Search
http://www.songlyrics.com/machine-gun-kelly-feat-camila-cabello/bad-things-lyrics/
http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=Zayn+%2F+Taylor+Swift+I+Don%27t+Wanna+

In [63]:
for song in songs:
    if song['lyrics'] is None:
        song['lyrics'] = get_song_lyrics(song['artist'], song['title'])
        print(song)

http://www.songlyrics.com/index.php?section=search&searchIn1=artist&searchIn3=song&searchW=Starley+Call+On+Me&submit=Search
http://www.songlyrics.com/starley/call-on-me-lyrics/
{'lyrics': "When you're low\nAnd your knees can't rise\nYou feel helpless\nAnd you're looking to the sky\nSome people would say\nTo accept their fate\nWell, if this is fate\nThen we'll find a way to cheat\n\nCause, oh, we'll say a little prayer\nBut, oh, if the answer isn't there\n\nYou know you can call on me\nWhen you need somebody\nYou know you can call on me\nWhen you can't stop the tears from falling down\nYou know you can call on me\nCall on me, darling\nYou know you can call on me\nCall on me, darling\n\nWhen you're weary\nAnd the road is dark\nAnd I'll guide you\nWith the beating of my heart\nAnd if the calvary\nAnd the help don't come\nWell, then we'll find a way\nTo dodge a smoking gun\n\nCause, oh, we'll say a little prayer\nBut, oh, if the answer isn't there\n\nYou know you can call on me\nWhen you n

# Putting it all together

1. Scrape hot 100 billboard songs
2. Obtain lyrics for each song
3. Save song list to csv file
4. Save lyrics to text files

In [17]:
from slugify import slugify
import os
import time
import csv

def create_dirs():
    dir_path = os.path.dirname(os.path.realpath('.'))
    data_dir = os.path.join(dir_path, 'billboard-hot-100-data')
    save_dir = os.path.join(data_dir, time.strftime("%d-%m-%Y"))
    
    os.makedirs(save_dir, exist_ok=True)
    
    return save_dir


def save_lyrics(save_dir, artist, title, lyrics):
    slugged = slugify(artist + ' ' + title, only_ascii=True) + '.txt'
    save_path = os.path.join(save_dir, slugged)
    
    try:
        f = open(save_path, 'w')
        f.write(lyrics)
    except:
        save_path = ''
    
    return save_path
    

def save_hot_100_and_lyrics():
    
    # create directories to save data to
    save_dir = create_dirs()
    
    # set up csv writer to write songs to
    hot_100_csv_path = os.path.join(save_dir, 'songs.csv')
    hot_100_csv = open(hot_100_csv_path, 'w')
    csv_writer = csv.writer(hot_100_csv, dialect='excel')
    csv_writer.writerow(['artist', 'title', 'rank_this_week', 'rank_last_week', 'lyrics_path'])
    
    for song in get_billboard_hot_100():
        
        # first attempt to retrieve song from SongLyrics
        lyrics = get_song_lyrics(song['artist'], song['title'])
        
        # try LyricsFreak
        if lyrics is None:
            lyrics = get_lyrics_freak(primary_artist(song['artist']), song['title'])
        
        if lyrics is None:
            raise "Unable to fetch lyrics for: " + song['artist'] + " " + song['title']
            
        # save lyrics to file and write song to csv
        lyric_file_path = save_lyrics(save_dir, song['artist'], song['title'], lyrics)
        csv_writer.writerow([
            song['artist'],
            song['title'],
            song['rank_this_week'],
            song['rank_last_week'],
            lyric_file_path
        ])
        
    hot_100_csv.close()
    
save_hot_100_and_lyrics()


http://www.songlyrics.com/index.php?searchW=Migos+Featuring+Lil+Uzi+Vert+Bad+And+Boujee&searchIn1=artist&submit=Search&section=search&searchIn3=song
http://www.songlyrics.com/migos/bad-and-boujee-feat-lil-uzi-vert-lyrics/
http://www.songlyrics.com/index.php?searchW=Ed+Sheeran+Shape+Of+You&searchIn1=artist&submit=Search&section=search&searchIn3=song
http://www.songlyrics.com/ed-sheeran/shape-of-you-lyrics/
http://www.songlyrics.com/index.php?searchW=The+Chainsmokers+Featuring+Halsey+Closer&searchIn1=artist&submit=Search&section=search&searchIn3=song
http://www.songlyrics.com/the-chainsmokers-feat-halsey/closer-feat-halsey-lyrics/
http://www.songlyrics.com/index.php?searchW=Machine+Gun+Kelly+x+Camila+Cabello+Bad+Things&searchIn1=artist&submit=Search&section=search&searchIn3=song
http://www.songlyrics.com/machine-gun-kelly-feat-camila-cabello/bad-things-lyrics/
http://www.songlyrics.com/index.php?searchW=Zayn+%2F+Taylor+Swift+I+Don%27t+Wanna+Live+Forever+%28Fifty+Shades+Darker%29&searchIn